In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
data = pd.read_csv('data/peliculas_ETL.csv', low_memory=False, index_col='id')
df = data.copy()
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42196 entries, 862 to 461257
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  42196 non-null  object 
 1   budget                 42196 non-null  float64
 2   genres                 42196 non-null  object 
 3   original_language      42196 non-null  object 
 4   overview               42196 non-null  object 
 5   popularity             42196 non-null  float64
 6   production_companies   42196 non-null  object 
 7   production_countries   42196 non-null  object 
 8   release_date           42196 non-null  object 
 9   revenue                42196 non-null  float64
 10  runtime                42196 non-null  float64
 11  spoken_languages       42196 non-null  object 
 12  status                 42196 non-null  object 
 13  tagline                42196 non-null  object 
 14  title                  42196 non-null  object 
 15  vote

Eliminemos la columna fecha, ya que no nos va a servir.

In [42]:
df.drop('release_date', axis=1, inplace=True)

separemos las columnas tipo categoricas de las numericas.

In [43]:
columnas_df = ['belongs_to_collection', 'budget', 'genres', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'release_year', 'return']

In [44]:
cols_numeric = [columna for columna in columnas_df if df[columna].dtype in ['int64', 'float64']]
cols_categoric = [columna for columna in columnas_df if df[columna].dtype == 'O']
df_numeric = df[cols_numeric]
df_categoric = df[cols_categoric]

Observemos un poco estos df

In [45]:
df_numeric.describe()

,budget,popularity,revenue,runtime,vote_average,vote_count,release_year,return
count,4.219600e+04,42196.000000,4.219600e+04,42196.000000,42196.000000,42196.000000,42196.000000,4.219600e+04
mean,4.160088e+06,2.914097,1.116670e+07,93.853302,5.631261,109.391720,1991.914399,7.006616e+06
std,1.721499e+07,5.744661,6.377178e+07,37.578123,1.921395,489.341742,23.839319,5.174193e+07
min,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1874.000000,-1.500000e+08
25%,0.000000e+00,0.387424,0.000000e+00,85.000000,5.000000,3.000000,1978.000000,0.000000e+00
50%,0.000000e+00,1.136089,0.000000e+00,95.000000,6.000000,10.000000,2001.000000,0.000000e+00
75%,0.000000e+00,3.717885,0.000000e+00,106.000000,6.800000,35.000000,2010.000000,0.000000e+00
max,3.800000e+08,547.488300,2.787965e+09,1256.000000,10.000000,14075.000000,2020.000000,2.550965e+09


In [46]:
df_categoric.describe()

,belongs_to_collection,genres,original_language,overview,production_companies,production_countries,spoken_languages,status,tagline,title
count,42196,42196,42196,42196,42196,42196,42196,42196,42196,42196
unique,1678,3857,89,41138,21141,2246,1755,6,18941,42196
top,N/D,['Drama'],en,N/D,N/D,['United States of America'],['English'],Released,N/D,Toy Story
freq,37840,4563,30012,902,11010,16708,20837,41866,23157,1
